7001 - Lab 1: Introduction to NLP with Keras

1 - Starting up with Python and Keras

1.2 - Introduction to Keras

In [1]:
# from keras import backend
from tensorflow.keras import backend
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
from tensorflow.keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
   loss='mse',
   metrics=['accuracy'])


In [0]:
model.fit(input_tensor, target_tensor, bach_size=128, epochs=10)

NameError: ignored

2 - Tensors in Keras

Scalars: 3 

In [3]:
import numpy as np

x = np.array(3)
x

array(3)

In [4]:
x.ndim

0

Vectors: [1,2,3]

In [5]:
x = np.array([1,2,3])
x

array([1, 2, 3])

In [6]:
x.ndim

1

Matrices: images

In [7]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

digit = train_images[4]

11493376/11490434 [==============================] - 7s 1us/step


In [8]:
digit.ndim

2

In [9]:
import matplotlib.pyplot as plt
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()

<Figure size 640x480 with 1 Axes>

In [10]:
digit.shape

(28, 28)

In [11]:
digit.dtype

dtype('uint8')

3D tensors: MNIST training data

In [12]:
print(train_images.ndim)

3


In [13]:
print(train_images.shape)

(60000, 28, 28)


In [14]:
print(train_labels.ndim)

1


In [15]:
print(train_labels.shape)

(60000,)


Batches and slices

In [16]:
batch = train_images[:128]
batch.shape

(128, 28, 28)

In [20]:
batch = train_images[128:256]
n=1

In [21]:
batch = train_images[128*n:128*(n+1)]

In [22]:
batch = train_images[:, 14:, 14:]

2.4 Tensor operations

In [23]:
x= np.array([1.,-1.,2.])
y = np.array([2.,4.,1.])
z = x+y 
print(z)

[3. 3. 3.]


In [24]:
z.ndim

1

In [25]:
z = np.maximum(x, 0.)
print(z)

[1. 0. 2.]


Tensor dot vs tensor multiplication

In [26]:
z = x*y
print(z)

[ 2. -4.  2.]


In [27]:
z = np.dot(x,y)
print(z)

0.0


Reshaping

In [28]:
w = np.array([[1,2,3,4],
              [5,6,7,8]])
print(w)

[[1 2 3 4]
 [5 6 7 8]]


In [29]:
w.shape

(2, 4)

In [30]:
w = np.transpose(w)
w.shape

(4, 2)

In [31]:
print(w)

[[1 5]
 [2 6]
 [3 7]
 [4 8]]


In [32]:
x = np.array([[0., 1.],
              [2., 3.],
              [4., 5.]])
print(x.shape)

(3, 2)


In [33]:
x = x.reshape((6, 1))
x

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

3.  An example of binary classification in Keras: classifying movie reviews

3.1 The IMDB dataset

In [2]:
from tensorflow.keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) =imdb.load_data(num_words=10000)

train_data and test_data are lists of reviews

In [3]:
train_data

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

whereas train_labels and test_labels are lists of 0s and 1s

In [4]:
train_labels

array([1, 0, 0, ..., 0, 1, 0])

In [5]:
train_labels[0]

1

3.2 Vectorizing the IMDB data

Extracting one-hot encodings of documents from their list encoding

In [6]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

In [14]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

Vectorizing the train and test labels

In [42]:
# Our vectorized labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

3.3 Defining the model

The architecture

In [44]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Loss function and optimizer

In [45]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

3.4 Training

In [46]:
model.fit(x_train, y_train, epochs=4, batch_size=512)

Train on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 43s 2ms/sample - loss: 0.4443 - accuracy: 0.8264
Epoch 2/4
25000/25000 [==============================] - 2s 97us/sample - loss: 0.2551 - accuracy: 0.9106
Epoch 3/4
25000/25000 [==============================] - 1s 59us/sample - loss: 0.1993 - accuracy: 0.9296
Epoch 4/4
25000/25000 [==============================] - 1s 59us/sample - loss: 0.1665 - accuracy: 0.9394


3.5 Testing

In [47]:
results = model.evaluate(x_test, y_test)

25000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [48]:
results

[0.3121018247127533, 0.87764]

In [49]:
model.predict(x_test)

array([[0.17401621],
       [0.9996499 ],
       [0.7093152 ],
       ...,
       [0.09528673],
       [0.05047736],
       [0.37528694]], dtype=float32)